<a href="https://colab.research.google.com/github/mattdhill011/big-data-challenge/blob/main/apparel_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Shamelessly copied from one of our in-class assignments


import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,965 kB]
Hit:13 http://ppa.la

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NLPTokens").getOrCreate()

In [3]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [4]:
from pyspark import SparkFiles

apparel = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(apparel)

apparel_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Apparel_v1_00.tsv.gz"), inferSchema=True, sep='\t', header=True, timestampFormat="yyyy-mm-dd")

In [5]:
apparel_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...|2013-01-14 00:01:00|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...| 

In [6]:
count = apparel_df.count()

print(f"There are {count} rows in the dataset")

There are 5906333 rows in the dataset


In [7]:
# We use the printSchema() method to make sure that the table schema matches what we want in the shema file
# this seems to be the case, so we don't have to do much more ETL than just selecting the relevant columns

review_id_table = apparel_df.select(["review_id","customer_id","product_id","product_parent","review_date"])

review_id_table.printSchema()
review_id_table.show()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: timestamp (nullable = true)

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R1KKOXHNI8MSXU|   32158956|B01KL6O72Y|      24485154|2013-01-14 00:01:00|
|R26SP2OPDK4HT7|    2714559|B01ID3ZS5W|     363128556|2014-01-04 00:03:00|
| RWQEDYAX373I1|   12608825|B01I497BGY|     811958549|2015-01-12 00:07:00|
|R231YI7R4GPF6J|   25482800|B01HDXFZK6|     692205728|2015-01-03 00:06:00|
|R3KO3W45DD0L1K|    9310286|B01G6MBEBY|     431150422|2015-01-12 00:06:00|
|R1C4QH63NFL5NJ|   26631939|B01FWRXN0Y|     366144407|2014-01-22 00:04:00|
|R2GP65O1U9N7BP|   48785098|B01EXNH1HE|     786052021|2015-01-28 00:07:00|
|R3O29CT5MQQ3XQ|   39548

In [9]:
products = apparel_df.select(["product_id", "product_title"]).distinct()
products.printSchema()
products.show()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|9822497415|Tommy Bahama Bird...|
|B00006AW85|John Deere Sweats...|
|B00006LFGJ|Women's Black Fle...|
|B00006THQ5|Roberto Coin Star...|
|B00006WMIV|Microfiber Contou...|
|B00006WRFU|Kenneth Cole Reac...|
|B00006X1WG|Undergear Cotton ...|
|B00006XJ1C|Silk Noile Trouse...|
|B00007EUVW|LEVIS SALT STAIN ...|
|B00007GBIJ|        Custom Rolex|
|B00007ID05|Deluxe Arctic Sur...|
|B0000863G5|108 Fgh40 Ecru Le...|
|B00008IOAK|Braza Reveal Clot...|
|B00008NRDU|           SWEET TEE|
|B00008NSOG|Reaction Kenneth ...|
|B00008ZGCO| UTILITY CARGO CAPRI|
|B00009KEIJ|Women's Mossimo E...|
|B00009YRTH|Thermal Crewneck ...|
|B0000ASFXX|Carhartt Men's Qu...|
|B0000CC55O|Youth M A1 Jacket...|
+----------+--------------------+
only showing top 20 rows



In [12]:
# I'm not 100% sure what the customer_count column is supposed to be, 
# so I assumed that it is looking for the count of how often each customer appears in the dataset

from pyspark.sql.functions import countDistinct

# First we group by customer_id, the for each group we use the countDistinct function to count how many distinct reviews were left by each cutsomer
# Then using the withColumnRenamed method we rename the column from the default (count(review_id)) to the "customer_count" name we want
customers = apparel_df.groupBy("customer_id").agg(countDistinct('review_id')).withColumnRenamed("count(review_id)", "customer_count")
customers.printSchema()
customers.show()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|      10206|             1|
|      15957|            10|
|      18051|             4|
|      18979|             1|
|      19079|             2|
|      20497|             1|
|      22373|             1|
|      26087|             1|
|      28088|             1|
|      28759|             1|
|      31367|             1|
|      32396|             2|
|      32855|             1|
|      38422|             1|
|      41409|             1|
|      51123|             2|
|      53963|             1|
|      60769|             1|
|      63155|             1|
|      69352|             2|
+-----------+--------------+
only showing top 20 rows



In [13]:
vine_table = apparel_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.printSchema()
vine_table.show()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1KKOXHNI8MSXU|          4|            0|          0|   N|
|R26SP2OPDK4HT7|          5|            1|          2|   N|
| RWQEDYAX373I1|          5|            0|          0|   N|
|R231YI7R4GPF6J|          5|            0|          0|   N|
|R3KO3W45DD0L1K|          5|            0|          0|   N|
|R1C4QH63NFL5NJ|          5|            0|          0|   N|
|R2GP65O1U9N7BP|          5|            0|          0|   N|
|R3O29CT5MQQ3XQ|          4|            0|          0|   N|
|R1ZECD2AA8QFF6|          5|            0|          0|   N|
|R2S79GCF6J89OA|          3|            0|          0|   N|
|R1